In [21]:
#Melt
import pandas as pd
df = pd.read_csv('C:\\Users\\joshu\\OneDrive\\Desktop\\python\\Intro to programming\\diabetic_data.csv') #load dataframe I am using the diabetic_data for my dataframe

# Identify the diagnosis columns
diag_cols = [c for c in ['diag_1', 'diag_2', 'diag_3'] if c in df.columns]
id_vars = [c for c in df.columns if c not in diag_cols] #all other columns stay as identifiers

melted = df.melt(id_vars=id_vars, value_vars=diag_cols,
                 var_name='diag_number', value_name='diagnosis')

melted = melted.dropna(subset=['diagnosis']).reset_index(drop=True) #drop rows where diagnosis is NaN
print(melted.head(10))

   encounter_id  patient_nbr             race  gender       age weight  \
0       2278392      8222157        Caucasian  Female    [0-10)      ?   
1        149190     55629189        Caucasian  Female   [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female   [20-30)      ?   
3        500364     82442376        Caucasian    Male   [30-40)      ?   
4         16680     42519267        Caucasian    Male   [40-50)      ?   
5         35754     82637451        Caucasian    Male   [50-60)      ?   
6         55842     84259809        Caucasian    Male   [60-70)      ?   
7         63768    114882984        Caucasian    Male   [70-80)      ?   
8         12522     48330783        Caucasian  Female   [80-90)      ?   
9         15738     63555939        Caucasian  Female  [90-100)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                       

In [20]:
#PIVOT
#required columns
assert 'time_in_hospital' in df.columns and 'race' in df.columns and 'gender' in df.columns
#pivot table with mean and also count
pivot = df.pivot_table(index = 'race', columns='gender',
                       values='time_in_hospital',
                       aggfunc=['mean', 'count'])
#change column names to be simple
pivot.columns = [f"{stat}_{gender}" for stat, gender in pivot.columns]
pivot = pivot.reset_index()

print(pivot.head())

              race  mean_Female  mean_Male  mean_Unknown/Invalid  \
0                ?     4.429832   4.149385                   4.5   
1  AfricanAmerican     4.541269   4.455493                   NaN   
2            Asian     3.899371   4.089783                   NaN   
3        Caucasian     4.481443   4.281379                   NaN   
4         Hispanic     4.086081   4.029630                   NaN   

   count_Female  count_Male  count_Unknown/Invalid  
0        1133.0      1138.0                    2.0  
1       11728.0      7482.0                    NaN  
2         318.0       323.0                    NaN  
3       39689.0     36410.0                    NaN  
4        1092.0       945.0                    NaN  


In [19]:
#Aggregation
#first choose a grouping column
group_col = 'admission_type_id'
assert group_col in df.columns

agg_summary = df.groupby(group_col).agg(
    patient_count = ('patient_nbr', 'nunique') if 'patient_nbr' in df.columns else ('time_in_hospital', 'count'), #nunique counts distinct patients useful if patient_nbr repeats
    avg_time_in_hospital = ('time_in_hospital', 'mean'),
    median_num_medications = ('num_medications', 'median'),
    avg_lab_procs = ('num_lab_procedures', 'mean')
).reset_index()

print(agg_summary)


   admission_type_id  patient_count  avg_time_in_hospital  \
0                  1          39879              4.378274   
1                  2          15367              4.610714   
2                  3          16325              4.320473   
3                  4             10              3.200000   
4                  5           3364              3.947126   
5                  6           4697              4.583066   
6                  7             21              4.857143   
7                  8            318              3.062500   

   median_num_medications  avg_lab_procs  
0                    14.0      47.245286  
1                    14.0      40.857955  
2                    17.0      34.971859  
3                    12.5      46.200000  
4                    15.0      26.766144  
5                    15.0      52.201663  
6                    19.0      47.857143  
7                    16.0      44.428125  


In [18]:
#iteration
#summarize first 5 rows
for i, row in enumerate(df.itertuples(index=False), 1):
    print("Row", i, "| patient_nbr:", getattr(row, 'patient_nbr', None),
          "| age:", getattr(row, 'age', None),
          "| time_in_hospital:", getattr(row, 'time_in_hospital', None))
    if i >= 5:
        break


Row 1 | patient_nbr: 8222157 | age: [0-10) | time_in_hospital: 1
Row 2 | patient_nbr: 55629189 | age: [10-20) | time_in_hospital: 3
Row 3 | patient_nbr: 86047875 | age: [20-30) | time_in_hospital: 2
Row 4 | patient_nbr: 82442376 | age: [30-40) | time_in_hospital: 2
Row 5 | patient_nbr: 42519267 | age: [40-50) | time_in_hospital: 1


In [ ]:
#Groupby
#multi-aggregation by 'age' and 'diabetesMed'
group_col = ['age', 'diabetesMed']
assert all(c in df.columns for c in group_col)

grouped = df.groupby(group_col).agg(
    patient_count = ('patient_nbr', 'nunique') if 'patient_nbr' in df.columns else ('time_in_hospital', 'count'),
    avg_lab_procs = ('num_lab_procedures', 'mean'),
    avg_num_med = ('num_medications', 'mean')
).reset_index()

print(grouped.head())
#try something for new  
# using transform to add a column with group's mean num_medications
if 'num_medications' in df.columns:
    df['group_avg_num_medications'] = df.groupby(group_col)['num_medications'].transform('mean')

#using filter to keep only groups with at least 50 patients
if 'patient_nbr' in df.columns:
    large_groups = df.groupby(group_col).filter(lambda g: g['patient_nbr'].nunique() >= 50) #keeps only rows from groups that satisfy the condition of group sizes > 50
    print("Rows in large groups:", large_groups.shape[0])

       age diabetesMed  patient_count  avg_lab_procs  avg_num_med
0   [0-10)          No             26      33.464286     4.250000
1   [0-10)         Yes            130      42.601504     6.586466
2  [10-20)          No             87      39.373626     7.296703
3  [10-20)         Yes            467      43.661667     8.426667
4  [20-30)          No            284      38.722222    10.906433
Rows in large groups: 101738
